In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests as req
import tempfile as tmpf
import numpy as np

In [2]:
uk_url = 'https://api.coronavirus.data.gov.uk/v1/data?filters=areaName=United%2520Kingdom;areaType=overview&structure=%7B%22areaType%22:%22areaType%22,%22areaName%22:%22areaName%22,%22areaCode%22:%22areaCode%22,%22date%22:%22date%22,%22newCasesByPublishDate%22:%22newCasesByPublishDate%22,%22cumCasesByPublishDate%22:%22cumCasesByPublishDate%22%7D&format=csv'


In [3]:
with tmpf.NamedTemporaryFile(mode='w') as f:
    response = req.get(uk_url)
    f.write(response.text)
    data = pd.read_csv(f.name, parse_dates=['date'])

data = data.loc[:, ['date',  'newCasesByPublishDate']].sort_values('date', ascending=True)
data['daily_cases_mean'] = data.newCasesByPublishDate.rolling(7).mean()
data['increase'] = data.newCasesByPublishDate.pct_change()
data = data.set_index('date').sort_index().loc['2020-03-05':]


In [63]:
import ipywidgets as widgets
import datetime

In [62]:
def f(starting_date):
    plt.style.use('seaborn')
    fig, (ax2, ax) = plt.subplots(nrows=2, figsize=(10, 12))
    data.daily_cases_mean.pct_change().rolling(7).mean().loc[starting_date:].plot(ax=ax, label='Daily percentage increase (7 days avg)');
    ax.axhline(0, color='red', label='Linear growth');
    ax.axhline(0.1, color='red', linestyle=':', label='7 days doubling rate');
    ax.legend();
#     ax.get_xaxis().set_visible(False)
    ax.set_title('Growth rate')
    ax2.set_title('Daily new cases')
    ax2.bar(x=data.index[data.index.slice_indexer(starting_date)], 
           height='newCasesByPublishDate', data=data.loc[starting_date:])
    ax2.plot('daily_cases_mean', data=data.loc[starting_date:], c='red', label='7 day rolling average');
    ax2.legend()
    
slider = widgets.SelectionSlider(options=data.index.date, continuous_update=False, value=datetime.date(2020, 8, 15),
                        readout=True, description='Starting date',
                                layout=widgets.Layout(width='80%'))
    
widgets.interact(f, starting_date=slider);

interactive(children=(SelectionSlider(continuous_update=False, description='Starting date', index=163, layout=…

In [18]:
data['increase'] = data.daily_cases_mean.pct_change()
data['increase_mean'] = data.increase.rolling(7).mean()

renamings = {'newCasesByPublishDate': 'Daily New Cases', 
             'daily_cases_mean': 'Daily New Cases (7 day avg.)', 
             'increase': 'Daily Cases Avg % Change', 'increase_mean': 'Daily Cases Avg % Change (7 day avg.)'}

data.rename(renamings, axis=1).tail(21)

,Daily New Cases,Daily New Cases (7 day avg.),Daily Cases Avg % Change,Daily Cases Avg % Change (7 day avg.)
date,,,,
2020-09-01,1295,1338.571429,0.011988,0.032476
2020-09-02,1508,1404.285714,0.049093,0.035002
2020-09-03,1735,1434.714286,0.021668,0.031828
2020-09-04,1940,1529.571429,0.066116,0.036981
2020-09-05,1813,1630.285714,0.065845,0.049474
2020-09-06,2988,1812.142857,0.111549,0.055683
2020-09-07,2948,2032.428571,0.121561,0.063974
2020-09-08,2460,2198.857143,0.081887,0.073960
2020-09-09,2659,2363.285714,0.074779,0.077629
